In [3]:
#@title measure time
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-2 --epoch_id 1700 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 100 --measure_time --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

16 [1, 2, 2] [32]
GEN: <class 'score_sde.models.ncsnpp_generator_adagn.NCSNpp'>
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
Working with z of shape (1, 4, 16, 16) = 1024 dimensions.
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since

In [4]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-2 --epoch_id 1700 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

16 [1, 2, 2] [32]
GEN: <class 'score_sde.models.ncsnpp_generator_adagn.NCSNpp'>
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
Working with z of shape (1, 4, 16, 16) = 1024 dimensions.
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since

In [6]:
!python3 test_iddgan_celeba.py --dataset afhq_cat --exp vq-f4-256 --epoch_id 400 --num_channels 3 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 100 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 2 --image_size 256 --current_resolution 64 --attn_resolutions 16 \
--scale_factor 6.0 --AutoEncoder_config autoencoder/config/vq-f4.yaml --AutoEncoder_ckpt autoencoder/weight/vq-f4.ckpt \
--batch_size 50 --measure_time --compute_fid --real_img_dir data/afhq/train/cat

64 [1, 2, 2, 2] [16]
GEN: <class 'score_sde.models.ncsnpp_generator_adagn.NCSNpp'>
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
loaded pretrained LPIPS loss from taming/modu

In [ ]:
dataset: CIFAR10, exp: kl-f2, epoch:475, FID: 9.471584868250147

In [ ]:
dataset: cifar10, exp: kl-f2-2, epoch: 1600, FID: 4.540297830066777

In [ ]:
dataset: cifar10, exp: kl-f2-2, epoch: 1700, FID: 4.3555737228003295, Inference time: 564.31+/-62.22ms

In [ ]:
AFHQ-CAT vq-f4-256 epoch:400 FID = 22.459739544399042

In [ ]:
AFHQ-CAT vq-f4-256 epoch:500 FID = 21.64908864472619